In [2]:
from app.db import connectSQL
import pandas as pd
from sqlalchemy import bindparam, text, String, Date, Integer
import numpy as np

con = connectSQL() 

#matchID = 1
#stmt = text("EXEC [Game].[procNationalTeamsShowAll] @MatchYearId = :matchID")
stmt = text("EXEC [Game].[procNationalTeamsShowAll]")

#stmt = stmt.bindparams(
#                bindparam('matchID', value=matchID, type_=Integer)
#            )

res = pd.read_sql_query(stmt, con)

res.to_csv('FifaWC2022/NationalTeamsMappingIDs.csv', index=False)

Connecting SQL...


In [9]:
natTeams = pd.DataFrame(res)
natTeams['NationalTeamNameLower'] = natTeams['NationalTeamName'].str.lower()
natTeams

,NationalTeamId,NationalTeamName,NationalTeamNameLower
0,190,AFGHANISTAN,afghanistan
1,76,ALBANIA,albania
2,117,ALGERIA,algeria
3,222,AMERICAN SAMOA,american samoa
4,193,ANDORRA,andorra
...,...,...,...
219,75,WALES,wales
220,42,YEMEN,yemen
221,131,ZAMBIA,zambia
222,209,ZANZIBAR,zanzibar


In [10]:
import pandas as pd
import numpy as np
from app.functions import extractDataset, Teams
team = []
teamlist = extractDataset('FifaWC2022/linkdata.csv')
numdata = len(teamlist)
for i in range(numdata):
    t = Teams(i+1,teamlist.index[i],teamlist.iloc[i]["link"])    
    team.append(t)
a=dict(team[1])
a['team head'][0]['Confederation']

m=team[1].getTeamsDetailsSoup()
m = m.loc[:, m.columns != 'teamid']
#df1['tilte'] = df1['category_id'].map(df2.set_index('id')['title'])
m['countryLower'] = m['Country'].str.lower()
m['countryid'] = m['countryLower'].map(natTeams.set_index('NationalTeamNameLower')['NationalTeamId'])
m

,Number,Position,Player name,Date of birth,Caps,Goals,Current club,Country,countryLower,countryid
playerid,,,,,,,,,,
1,1,GK,Alireza Beiranvand,1992-09-21,53,0,Persepolis,Iran,iran,17
2,12,GK,Payam Niazmand,1995-04-06,1,0,Sepahan,Iran,iran,17
3,22,GK,Amir Abedzadeh,1993-04-26,11,0,Ponferradina,Spain,spain,70
4,24,GK,Hossein Hosseini,1992-06-30,8,0,Esteghlal,Iran,iran,17
5,2,DF,Sadegh Moharrami,1996-03-01,22,0,Dinamo Zagreb,Croatia,croatia,47
6,3,DF,Ehsan Hajsafi (captain),1990-02-25,123,7,AEK Athens,Greece,greece,54
7,4,DF,Shojae Khalilzadeh,1989-05-14,25,1,Al-Ahli,Qatar,qatar,28
8,5,DF,Milad Mohammadi,1993-09-29,47,1,AEK Athens,Greece,greece,54
9,8,DF,Morteza Pouraliganji,1992-04-19,48,3,Persepolis,Iran,iran,17


In [32]:
import json
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
data = []
data1 = []
url = "https://www.countries-ofthe-world.com/flags-of-the-world.html"
page = requests.get(url)
soup = BeautifulSoup(page.text, parser='lxml')

#table = soup.select('table', attrs={"class":"sortable wikitable plainrowheaders jquery-tablesorter"})
table = soup.find( "table", {"class":"two-column td-image"} )
table_body = table.find('tbody')

rows = table_body.find_all('tr')
for row in rows:
    cols = row.find_all('td') 
    
    t = row.find_all('img')
    cols1 = [ele.get('src') for ele in t]       
    #print(row['a','title'])
    cols = [ele.text.strip() for ele in cols]
    datas1 = [ele for ele in cols]
    datas2 = [ele for ele in cols1]
    res = [*datas1, *datas2]
    data.append(res) # Get rid of empty values   
    #data1.append([ele for ele in cols1]) # Get rid of empty values    

tab1 = pd.DataFrame(data)
tab1.columns = tab1.iloc[0]
tab1 = tab1[1:]
tab1 = tab1.dropna()
ColumnNames = {"A":"Country",None:"FlagPath"}            
tab1 = tab1.rename(columns=ColumnNames)
tab1['FlagPath'] = tab1['FlagPath'].apply(lambda x: "{}{}".format('https://www.countries-ofthe-world.com/', x))
tab1['CountryLower'] = tab1['Country'].str.lower()
tab1['countryid'] = tab1['CountryLower'].map(natTeams.set_index('NationalTeamNameLower')['NationalTeamId'])
#dtypescols = {"countryid":"int"}
#tab1 = tab1.astype(dtypescols)
tab1['countryid'] = tab1['countryid'].fillna(0)
tab1['countryid'] = tab1['countryid'].astype(int)
tab1.loc[:4]

,,Country,FlagPath,CountryLower,countryid
1,,Afghanistan,https://www.countries-ofthe-world.com/flags-no...,afghanistan,190
2,,Albania,https://www.countries-ofthe-world.com/flags-no...,albania,76
3,,Algeria,https://www.countries-ofthe-world.com/flags-no...,algeria,117
4,,Andorra,https://www.countries-ofthe-world.com/flags-no...,andorra,193


In [86]:
import json
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
data = []
data1 = []
url = "https://en.wikipedia.org/wiki/Costa_Rica_national_football_team"
page = requests.get(url)
soup = BeautifulSoup(page.text, parser='lxml')

#table = soup.select('table', attrs={"class":"sortable wikitable plainrowheaders jquery-tablesorter"})
table = soup.find( "table", {"class":"plainrowheaders"} )
table_body = table.find('tbody')

rows = table_body.find_all('tr')
for row in rows:
    cols = row.find_all(['td','th']) 
    
    t = row.find_all('img')
    cols1 = [ele.get('alt') for ele in t]       
    #print(row['a','title'])
    cols = [ele.text.strip() for ele in cols]
    datas1 = [ele for ele in cols]
    datas2 = [ele for ele in cols1]
    res = [*datas1, *datas2]
    data.append(res) # Get rid of empty values   
    #data1.append([ele for ele in cols1]) # Get rid of empty values    

tab1 = pd.DataFrame(data)
tab1.columns = tab1.iloc[0]
tab1 = tab1[1:]
tab1 = tab1.dropna()
tab1["Pos."] = tab1["Pos."].str[1:]
tab1["Date of birth (age)"] = tab1["Date of birth (age)"].str.replace('(', '', regex=True)
tab1["Date of birth (age)"] = tab1["Date of birth (age)"].str.partition(')')[0]
tab1["Date of birth (age)"] = pd.to_datetime(tab1["Date of birth (age)"])

tab1

,No.,Pos.,Player,Date of birth (age),Caps,Goals,Club,None
1,1,GK,Keylor Navas,1986-12-15,108,0,Paris Saint-Germain,France
2,18,GK,Esteban Alvarado,1989-04-28,25,0,Herediano,Costa Rica
3,23,GK,Patrick Sequeira,1999-03-01,2,0,Lugo,Spain
5,3,DF,Juan Pablo Vargas,1995-06-06,12,1,Millonarios,Colombia
6,4,DF,Keysher Fuller,1994-07-12,32,2,Herediano,Costa Rica
7,6,DF,Óscar Duarte,1989-06-03,72,4,Al-Wehda,Saudi Arabia
8,8,DF,Bryan Oviedo,1990-02-18,77,2,Real Salt Lake,United States
9,15,DF,Francisco Calvo,1992-07-08,76,8,Konyaspor,Turkey
10,16,DF,Carlos Martínez,1999-03-30,8,0,San Carlos,Costa Rica
11,19,DF,Kendall Waston,1988-01-01,64,9,Saprissa,Costa Rica


In [7]:
import json
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
data = []
url = "https://en.wikipedia.org/wiki/Costa_Rica_national_football_team"
page = requests.get(url)
soup = BeautifulSoup(page.text, parser='lxml')

#table = soup.select('table', attrs={"class":"sortable wikitable plainrowheaders jquery-tablesorter"})
table = soup.find( "table", {"class":"plainrowheaders"} )
table_body = table.find('tbody')

rows = table_body.find_all('tr')
for row in rows:
    cols = row.find_all(['td','th'])
    
    cols = [ele.text.strip() for ele in cols]
    
    data.append([ele for ele in cols]) # Get rid of empty values
    
tab1 = pd.DataFrame(data)
tab1 = tab1.dropna()
tab1.columns = tab1.iloc[0]
tab1 = tab1[1:]
tab1["Pos."] = tab1["Pos."].str[1:]
tab1["Date of birth (age)"] = tab1["Date of birth (age)"].str.replace('(', '', regex=True)
tab1["Date of birth (age)"] = tab1["Date of birth (age)"].str.partition(')')[0]
tab1["Date of birth (age)"] = pd.to_datetime(tab1["Date of birth (age)"])
tab1

,No.,Pos.,Player,Date of birth (age),Caps,Goals,Club
1,1,GK,Keylor Navas,1986-12-15,108,0,Paris Saint-Germain
2,18,GK,Esteban Alvarado,1989-04-28,25,0,Herediano
3,23,GK,Patrick Sequeira,1999-03-01,2,0,Lugo
5,3,DF,Juan Pablo Vargas,1995-06-06,12,1,Millonarios
6,4,DF,Keysher Fuller,1994-07-12,32,2,Herediano
7,6,DF,Óscar Duarte,1989-06-03,72,4,Al-Wehda
8,8,DF,Bryan Oviedo,1990-02-18,77,2,Real Salt Lake
9,15,DF,Francisco Calvo,1992-07-08,76,8,Konyaspor
10,16,DF,Carlos Martínez,1999-03-30,8,0,San Carlos
11,19,DF,Kendall Waston,1988-01-01,64,9,Saprissa


In [ ]:
import json
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
data = []
url = "https://en.wikipedia.org/wiki/Costa_Rica_national_football_team"
page = requests.get(url)
soup = BeautifulSoup(page.text, parser='lxml')

#table = soup.select('table', attrs={"class":"sortable wikitable plainrowheaders jquery-tablesorter"})
table = soup.find( "table", {"class":"plainrowheaders"} )
table_body = table.find('tbody')

rows = table_body.find_all('tr')
for row in rows:    
    cols = row.find_all(['td','th'])
    k= [col.text.strip() for col in cols if col]
    data.append(k)
data[1:]
sr = data[1:].copy()
sr[1][3]
sr = sr.replace("(","")
head, sep, tail = sr.partition(')')
sr = head
sr      


In [ ]:
sr = '(1986-12-15) 15 December 1986 (age\xa035)'
sr = sr.replace("(","")
sr = sr.partition(')')[0]
#sr = head
sr

In [ ]:
import json
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np


data = []
page = requests.get("https://en.wikipedia.org/wiki/Costa_Rica_national_football_team")
soup = BeautifulSoup(page.text, parser='lxml')
table = soup.find("table", {"class":"infobox"} )
table_body = table.find('tbody')
rows = table_body.find_all('tr')
for row in rows:
    cols = row.find_all(['th','td'])    
    cols = [ele.text.strip() for ele in cols if ele]    
    data.append(cols[0:2]) # Get rid of empty values    
tab1 = pd.DataFrame(data)
cols = {0:"ind",1:"val"}
columns = list(cols.values())
tab1 = tab1.rename(columns=cols)
tab1 = tab1.loc[:,columns]
tab1 = tab1.dropna()
datas = ["Head coach","FIFA code","Confederation","Captain"]
dat = tab1.loc[(tab1["ind"].isin(datas))]  
res = dat.set_index("ind").T
#res["teamid"] = self.teamid
res = res.to_dict(orient="records")

res


In [6]:
def append_to_excel(fpath, df, sheet_name):
    with pd.ExcelWriter(fpath, engine='openpyxl', mode='a', if_sheet_exists='replace') as f:
        df.to_excel(f, sheet_name=sheet_name, index=False) 

ass=dict(team[2])
ass = ass['team head']
ass = pd.DataFrame(ass)
ass['Team name'] = team[2].teamname
ass = ass.loc[:, ass.columns != 'teamid']
#teamname = pd.DataFrame(team[1].teamname)
dfNew=pd.concat([ass,m],axis=0, ignore_index=True)
append_to_excel('FifaWC2022/filesexport.xlsx', dfNew, team[1].teamname)

In [ ]:
import json
import requests
from bs4 import BeautifulSoup
def download_page(url):
    response = requests.get(url)
    response.raise_for_status()
    return response.text
def main(url):
    content = download_page(url)
    soup = BeautifulSoup(content, 'html.parser')
    result = {}
    for row in soup.table.find_all('thead'):
        row_header = row.th.get_text()
        row_cell = row.td.get_text()
        result[row_header] = row_cell
    with open('myCSV/book_table.json', 'w') as storage_file:
        storage_file.write(json.dumps(result))
if __name__ == "__main__":
    main("https://en.wikipedia.org/wiki/Costa_Rica_national_football_team")

In [ ]:
import pandas as pd
import numpy as np
table = pd.read_html("https://en.wikipedia.org/wiki/Costa_Rica_national_football_team")        

table        
cols = {0:"ind",1:"val"}
columns = list(cols.values())
tab = pd.DataFrame(table[0])
tab
tab = tab.rename(columns=cols)
tab = tab.loc[:,columns]
tab = tab.dropna()
datas = ["Head coach","FIFA code","Confederation"]
dat = tab.loc[(tab["ind"].isin(datas))]
res = dat.set_index("ind").T
res = res.to_dict(orient="records")
res[0]

In [ ]:
import pandas as pd
import numpy as np
table = pd.read_html("https://en.wikipedia.org/wiki/Costa_Rica_national_football_team")        

cols = {0:"ind",1:"val"}
columns = list(cols.values())
table[0]

In [ ]:
import pandas as pd
import numpy as np
from app.functions import extractDataset, Teams
team = []
teamlist = extractDataset('FifaWC2022/linkdata.csv')
numdata = len(teamlist)
for i in range(numdata):
    t = Teams(i+1,teamlist.index[i],teamlist.iloc[i]["link"])
    print(t.teamid, " - ", t.teamname, " - ", t.link)
    team.append(t)
    
for i in range(len(team)):
    
    print(team[i].teamname)
    print(team[i].getTeamsHead())
    #print(team[1].getTeamsHead())
#print(team[0].getTeamsDetails('No. Pos.'))
#print(team[1].getTeamsDetails('No. Pos.'))

In [ ]:
import pandas as pd
import numpy as np

table = pd.read_html("https://en.wikipedia.org/wiki/Ghana_national_football_team")
print(f'Total tables: {len(table)}')
table[3].info()

In [ ]:
import pandas as pd
import numpy as np
import requests

url = 'https://en.wikipedia.org/wiki/Costa_Rica_national_football_team'
html = requests.get(url).content

df_list = pd.read_html(html)
#pd.Series(['260,327,021']).str.replace(',', '').astype(int)
#df_list
#soup = BeautifulSoup(html)
#table = soup.select_one('table:contains("No. Pos.                 Player Date of birth (age)  Caps  Goals")')


In [ ]:
soup = BeautifulSoup("https://en.wikipedia.org/wiki/Australia_men%27s_national_soccer_team")
table = soup.select_one('table:contains("Current squad")')
table
#table1 = pd.read_html("https://en.wikipedia.org/wiki/Ghana_national_football_team", match='No.')
table1 = pd.read_html("https://en.wikipedia.org/wiki/Australia_men%27s_national_soccer_team", match='No.')

print(f'Total tables: {len(table1)}')
ColumnNames = {"No.":"Number","Pos.":"Position","Player":"Player name","Date of birth (age)":"Date of birth",
               "Club":"Current club"}
tab = pd.DataFrame(table1[6])
tab = tab.dropna()
tab = tab.rename(columns=ColumnNames)
dtypescols = {"Number":"int","Caps":"int","Goals":"int"}
tab = tab.astype(dtypescols)
tab.index = np.arange(1, len(tab) + 1)
tab.index.name = "test"
tab

In [ ]:
table = pd.read_html("https://en.wikipedia.org/wiki/Ghana_national_football_team")
print(f'Total tables: {len(table)}')
#cols = {0:"a",1:"b"}
tab = pd.DataFrame(table[8])
tab
#tab = tab.rename(columns=cols)
#tab = tab.loc[(tab["a"]=="Head coach")]["b"]
#tab.values[0]


In [ ]:
table = pd.read_html("https://en.wikipedia.org/wiki/Ghana_national_football_team")
print(f'Total tables: {len(table)}')
cols = {0:"ind",1:"val"}
tab = pd.DataFrame(table[0])
tab = tab.rename(columns=cols)
datas = ["Head coach","FIFA code","Confederation"]
dat = tab.loc[(tab["ind"].isin(datas))]
#dat = tab.loc[(tab["a"]=="Head coach") | (tab["a"]=="Confederation")]
#res = pd.DataFrame(dat["a"],dat["b"])
#dict(dat)
res = dat.set_index("ind").T
#res = dat.to_dict(orient="records")
res = res.to_dict(orient="records")
res

In [ ]:
table = pd.read_html("https://en.wikipedia.org/wiki/Ghana_national_football_team")
print(f'Total tables: {len(table)}')
cols = {0:"a",1:"b"}
tab = pd.DataFrame(table[0])
tab